In [1]:
import os

os.environ["BRASA_DATA_PATH"] = "D:\\brasa"

In [2]:
import sys
sys.path.append('..')

from datetime import datetime

import pandas as pd
import pyarrow.dataset as ds
import pyarrow.compute as pc
import pyarrow
from bizdays import Calendar, set_option

import brasa

man = brasa.CacheManager()

In [12]:
import numpy as np
def interp_ff(term, rates, terms):
    log_pu = np.log((1 + rates)**(terms/252))
    pu = np.exp(np.interp(term, terms, log_pu))
    return pu ** (252 / term) - 1

In [28]:
ds = brasa.get_dataset("b3-curves-dap")
df = ds.to_table().to_pandas()
df

,refdate,symbol,maturity_date,business_days,adjusted_tax
0,2010-01-04,DAPQ10,2010-08-15,155,0.04259
1,2010-01-04,DAPQ12,2012-08-15,659,0.06466
2,2010-01-04,DAPK15,2015-05-15,1349,0.06675
3,2010-01-05,DAPQ10,2010-08-15,154,0.04240
4,2010-01-05,DAPQ12,2012-08-15,658,0.06470
...,...,...,...,...,...
35068,2023-11-17,DAPQ40,2040-08-15,4208,0.05533
35069,2023-11-17,DAPK45,2045-05-15,5399,0.05424
35070,2023-11-17,DAPQ50,2050-08-15,6718,0.05643
35071,2023-11-17,DAPK55,2055-05-15,7911,0.05624


In [29]:
curve = ds.filter(pc.field("refdate") == datetime(2010, 1, 4)).to_table().to_pandas()

In [30]:
curve

,refdate,symbol,maturity_date,business_days,adjusted_tax
0,2010-01-04,DAPQ10,2010-08-15,155,0.04259
1,2010-01-04,DAPQ12,2012-08-15,659,0.06466
2,2010-01-04,DAPK15,2015-05-15,1349,0.06675


In [31]:
interp_ff(np.array([21, 126, 252]), curve["adjusted_tax"], curve["business_days"])

array([0.36049029, 0.05264656, 0.05364005])